In [22]:
root = '/content/drive/My Drive/nlp-lab'

#dataset = "Amazon EN"
dataset = "Amazon DE"

BERT_model_location = root + '/models/BERT-models/BERT-base-multilingual-cased/multi_cased_L-12_H-768_A-12'

In [23]:
drive.mount('/content/drive', force_remount=True)
path.append(root)

Mounted at /content/drive


In [24]:
if dataset == "Amazon EN":
  datasets_location = root + '/data/original-datasets/amazon-english-dataset/raw_data_english.json'
  processed_data_location = root + '/data/processed-data/amazon-english/processed_data_without_embeddings_no_outliers.json'

  data_df = pd.read_json(processed_data_location)
  raw_data_df = pd.read_json(datasets_location)

elif dataset == "Amazon DE":
  datasets_location = root + '/data/original-datasets/amazon-german-dataset2/raw_data_german2.json'
  processed_data_location = root + '/data/processed-data/amazon-german2/processed_data_without_embeddings_no_outliers.json'
  
  data_df = pd.read_json(processed_data_location)
  raw_data_df = pd.read_json(datasets_location)
  raw_data_df = raw_data_df.rename(columns={"review_body": "reviewText"})

In [6]:
!pip install bert-serving-server  # server
!pip install bert-serving-client  # client, independent of `bert-serving-server`
!pip install transformers

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 47.5MB/s 
     |████████████████████████████████| 3.0MB 42.4MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9aff5ca391b9a5ce9245a500927959a61eeea57760551db780cb85d4a6a73679
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [25]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
from transformers import BertTokenizer
import nltk
nltk.download('punkt')
import math
from google.colab import drive
from sys import path

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

In [27]:
get_ipython().system_raw('bert-serving-start -model_dir "$BERT_model_location" -num_worker=1 -max_seq_len=512 -cased_tokenization -pooling_strategy NONE -show_tokens_to_client | tee /tmp/output.txt &')

In [ ]:
#Test that BERT server is up (wait for some minutes after starting BERT server before running this cell. Otherwise the server crashes)
bc = BertClient()
print(bc.server_status)

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_sentence(sentence):
    return tokenizer.tokenize(sentence)

def get_token_embeddings(tokenized_sentence):
    tokenized_sentences_list = [tokenized_sentence]
    token_emeddings = bc.encode(tokenized_sentences_list, is_tokenized=True,show_tokens=True)
    return token_emeddings

def get_token_embeddings_from_sentence(sentence):
    tokenized_sentence = tokenize_sentence(sentence)
    token_embeddings = get_token_embeddings(tokenized_sentence)
    return token_embeddings

In [28]:
tokenized_comments = []
number_of_tokens_list = []
number_of_rows = raw_data_df.shape[0]

for index, row in raw_data_df.iterrows():
    
    if index%100 == 0:
        print('processed comments: ' + str(index) + '/' + str(number_of_rows) + 'percentage: ' + str(math.floor((index/number_of_rows)*100)) + '%')
    
    tokenized_comment = tokenize_sentence(row['reviewText'])
    tokenized_comments.append(tokenized_comment)
    
    number_of_tokens = len(tokenized_comment)
    number_of_tokens_list.append(number_of_tokens)
    
raw_data_df['number_of_tokens'] = number_of_tokens_list

processed comments: 1900/9899percentage: 19%
processed comments: 5900/9899percentage: 59%
processed comments: 6500/9899percentage: 65%
processed comments: 7000/9899percentage: 70%
processed comments: 8400/9899percentage: 84%
processed comments: 10100/9899percentage: 102%
processed comments: 11300/9899percentage: 114%
processed comments: 15100/9899percentage: 152%
processed comments: 19200/9899percentage: 193%
processed comments: 19400/9899percentage: 195%
processed comments: 27600/9899percentage: 278%
processed comments: 28300/9899percentage: 285%
processed comments: 30700/9899percentage: 310%
processed comments: 35800/9899percentage: 361%
processed comments: 37500/9899percentage: 378%
processed comments: 38300/9899percentage: 386%
processed comments: 38700/9899percentage: 390%
processed comments: 40500/9899percentage: 409%
processed comments: 42000/9899percentage: 424%
processed comments: 42100/9899percentage: 425%
processed comments: 44600/9899percentage: 450%
processed comments: 486

In [ ]:
raw_data_with_more_than_512_tokens_df = raw_data_df[raw_data_df['number_of_tokens'].apply(lambda x: x > 510)]
raw_data_df = raw_data_df[raw_data_df['number_of_tokens'].apply(lambda x: x <= 510)]
removed_comments = raw_data_with_more_than_512_tokens_df.index.unique()
data_df = data_df[data_df['comment_id'].apply(lambda x: x not in removed_comments)]

In [ ]:
assert(len(raw_data_df) == len(data_df['comment_id'].unique()))

In [29]:
embeddings_dimensionality = 768

def get_contextual_embeddings(comments_df):
    
    
    sentence_contextual_embeddings_array = np.ones([0,embeddings_dimensionality])
    #comment_counter = 0
    number_of_rows = raw_data_df.shape[0]
    
    for comment_index, comment_row in comments_df.iterrows():
        
        #if comment_counter%100 == 0:
            #print('processed comments: ' + str(comment_counter) + '/' + str(number_of_rows) + ' percentage: ' + str(math.floor((comment_counter/number_of_rows)*100)) + '%')
            
        #comment_counter += 1
        
        comment_token_embeddings = get_token_embeddings_from_sentence(comment_row['reviewText'])
        
        sentences_df = data_df[data_df['comment_id'].apply(lambda x: x == comment_index)]
        
        ending_index = 0
        
        for sentence_index, sentence_row in sentences_df.iterrows():
            
            tokenized_sentence = tokenize_sentence(sentence_row['sentence_text'])
            
            starting_index = ending_index + 1
            ending_index = starting_index + len(tokenized_sentence) - 1
            
            #embeddings_dimensionality = comment_token_embeddings[0].shape[2]
            sentence_token_embeddings = np.ones([len(tokenized_sentence), embeddings_dimensionality])
            
            i=0
            for token_index in range(starting_index, ending_index + 1):
                #print('i' + str(i))
                #print(token_index)
                sentence_token_embeddings[i] = comment_token_embeddings[0][0][token_index]
                i += 1
                
            sentence_contextual_embedding = np.reshape(np.mean(sentence_token_embeddings, axis=0),(1,embeddings_dimensionality))
            sentence_contextual_embeddings_array = np.append(sentence_contextual_embeddings_array,sentence_contextual_embedding, axis=0)

    return sentence_contextual_embeddings_array

In [ ]:
chunk_size= 100
checkpoint_number = 0

for i in range(0, len(raw_data_df), chunk_size):
  
  print('processing chunk: ' + str(checkpoint_number))
     
  chunk = raw_data_df[i:i+chunk_size]

  sentence_contextual_embeddings = get_contextual_embeddings(chunk)
  np.save('./checkpoints/contextual_embeddings' + str(checkpoint_number), sentence_contextual_embeddings)
  checkpoint_number += 1

In [ ]:
final_embeddings = np.ones([0,embeddings_dimensionality])

for i in range(0, checkpoint_number):
    checkpoint = np.load('./checkpoints/contextual_embeddings' + str(i) + '.npy')
    final_embeddings = np.append(final_embeddings,checkpoint, axis=0)

In [ ]:
assert(len(data_df) == len(final_embeddings))

data_with_embeddings = pd.concat([data_df.reset_index(drop=True),
                                  pd.DataFrame(final_embeddings)], axis=1)

In [ ]:
data_with_embeddings.to_json("\tmp\processed_data_with_CONTEXTUAL_embeddings_multilingual_cased_BERT_english.json")
#data_with_embeddings.to_json("\tmp\processed_data_with_CONTEXTUAL_embeddings_multilingual_cased_BERT_german2.json")